* Linear Regression
* Lasso Regression
* Ridge Regression
* Gradient Boosting Regression
* Random Forest Regression
* Support Vector Regression
* MLP Regression

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xgboost import XGBRegressor

from sklearn.preprocessing import StandardScaler,MinMaxScaler, MaxAbsScaler,RobustScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
# 1. 데이터 불러오기
data = pd.read_csv("C:/py_workspace/korea_py/prj_st/pt_train_dt.csv")
x_data = data.iloc[:,2:-4]
y_data = data.iloc[:,-4:-1]

In [3]:
# 2. 데이터 전처리
# 2.1. 결측치 처리

# 열별 결측치 확인
print('< 열별 결측치 확인 >\n')
print(x_data.isnull().sum(0))

# 결측치가 없는 열의 갯수
print(f'\n결측치가 없는 열의 갯수 : {np.array(x_data.isnull().sum(0)==0).sum()}\n')

# 행별 결측치 확인
print('\n< 행별 결측치 확인 >\n')
print(x_data.isnull().sum(1))

# 결측치가 없는 행의 갯수
print(f'\n결측치가 없는 행의 갯수 : {np.array(x_data.isnull().sum(1)==0).sum()}\n')

< 열별 결측치 확인 >

x_1       2596
x_2       2594
x_3       2594
x_4       2596
x_5       2594
x_6       2594
x_7       2596
x_8       2594
x_9       2594
x_10     10079
x_11     10080
x_12     10083
x_13     10083
x_14     10083
x_15     10084
x_16     10085
x_17     10087
x_18     10089
x_19     10092
x_20      2295
x_21      2534
x_22      2535
x_23      2536
x_24      2535
x_25      2537
x_26      2535
x_27      2535
x_28      2535
x_29      2541
x_30      4605
         ...  
x_371     4129
x_372     4132
x_373     4172
x_374     4092
x_375     4093
x_376     3730
x_377     3724
x_378    14149
x_379     3738
x_380     3724
x_381     3757
x_382     3757
x_383     3759
x_384     3757
x_385     3757
x_386     3758
x_387     6963
x_388     3722
x_389     3722
x_390      495
x_391      496
x_392      543
x_393      496
x_394      514
x_395      528
x_396      532
x_397      541
x_398      541
x_399      541
x_400      542
Length: 400, dtype: int64

결측치가 없는 열의 갯수 : 0


< 행별 결측치 확인 >

0       

==> 결측치가 포함된 열을 제거하는 로직보다는 결측치가 포함된 행을 제거하는 것이 좋아보임.

In [4]:
# (1) 결측치가 포함된 모든 행을 제거
x_drop_all = x_data.dropna(axis=0)
y_drop_all = y_data.loc[x_drop_all.index]

==> 결측치가 포함된 모든 행을 제거하여도 충분히 많은 관측치가 존재하므로 결측치 모두 제거하는 방식을 취함.  
만약 결측치를 대체하는 방식을 취하고 싶다면 아래 2 line 실행하면 됨.

==> 결측치 50개 이상 포함인 행 삭제로 결정

In [5]:
# 결측치 50개 이상 포함한 행 삭제
x_drop_50 = x_data.loc[x_data.isnull().sum(1)<=50]
y_drop_50  = y_data.loc[x_drop_50.index]

#   (1) 결측값을 0으로 채우기
x_drop_50_0 = x_drop_50.fillna(0)

#   (2) 결측값을 앞 방향 혹은 뒷 방향으로 채우기 
x_drop_50_f = x_drop_50.fillna(method='ffill') 
x_drop_50_b = x_drop_50.fillna(method='bfill')

#   (3) 결측값을 변수별 평균으로 대체
x_drop_50_m = x_drop_50.fillna(x_drop_50.mean())

#   (4) 결측값을 중앙값으로 대체
x_drop_50_med = x_drop_50.fillna(x_drop_50.median())

#   (5) 결측값을 최빈값으로 대체
x_drop_50_med = x_drop_50.fillna(x_drop_50.median())

In [6]:
# # 2.2. 데이터 살펴보기
# # correlation matrix
# corr_mat = x_drop_all.corr()
# corr_mat.style.background_gradient(cmap='coolwarm')

==> correlation matrix를 살펴 본 결과, 중복 열(correlation=1)의 존재 확인

In [7]:
# 중복 제거 전 data shape
print(f'중복 제거 전 : {x_drop_all.shape}')

#중복 행 제거
x_drop_all = x_drop_all.drop_duplicates()
y_drop_all = y_drop_all.loc[x_drop_all.index]
print(f'중복 행 제거 후 : {x_drop_all.shape}')

# 중복 열(변수) 제거
x_drop_all = x_drop_all.loc[:, ~x_drop_all.T.duplicated()]
y_drop_all = y_drop_all.loc[x_drop_all.index]
print(f'중복 열 제거 후 : {x_drop_all.shape}')

중복 제거 전 : (69175, 400)
중복 행 제거 후 : (69175, 400)
중복 열 제거 후 : (69175, 342)


==> 데이터를 살펴보았을 때 모든 행에서 같은 값을 지니는 의미없는 변수 군 파악했음.

In [8]:
# 의미없는 변수 제거
print(f'의미없는 변수 제거 전 : {x_drop_all.shape}')
x_drop_all = x_drop_all.loc[:,x_drop_all.std()!=0.0]
y_drop_all = y_drop_all.loc[x_drop_all.index]
print(f'의미없는 변수 제거 후 : {x_drop_all.shape}')

의미없는 변수 제거 전 : (69175, 342)
의미없는 변수 제거 후 : (69175, 341)


In [9]:
# # 2.3. 데이터 분포 살펴보기
# # 경로 생성
# os.makedirs('./histogram', exist_ok=True)
# os.makedirs('./boxplot', exist_ok=True)

# # 변수별 히스토그램
# for i in range(0,10):
#     temp_df = x_drop_all.iloc[:,i*35:35*(i+1)]
#     temp_df.hist(figsize=(35,35), layout=(7,5), bins=20)
#     plt.tight_layout()
#     plt.savefig(f"histogram/hist_{i}.png")

# # 변수별 박스플랏
# for i in range(0,10):
#     temp_df = x_drop_all.iloc[:,i*35:35*(i+1)]
#     temp_df.plot(kind="box",subplots=True,layout=(7,5),figsize=(15,15))
#     plt.tight_layout()
#     plt.savefig(f"boxplot/boxplot_{i}.png")

In [10]:
# 2.4. 아웃라이어 제거
# 이상치 제거 (1) : IQR 활용
def get_outlier_right_skewed(df=None, column=None, weight=1.0):
    quantile_1 = np.percentile(df[column].values, 5)
    quantile_2 = np.percentile(df[column].values, 95)
  
    IQR = quantile_2 - quantile_1
    IQR_weight = IQR * weight
  
    highest = quantile_2 + IQR_weight
  
    outlier_idx = df[column][(df[column] > highest)].index
    return outlier_idx

def get_outlier_no_skewed(df=None, column=None, weight=1.0):
    quantile_1 = np.percentile(df[column].values, 2.5)
    quantile_2 = np.percentile(df[column].values, 97.5)
  
    IQR = quantile_2 - quantile_1
    IQR_weight = IQR * weight
  
    lowest = quantile_1 - IQR_weight
    highest = quantile_2 + IQR_weight
  
    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    return outlier_idx

def get_outlier_left_skewed(df=None, column=None, weight=1.0):
    quantile_1 = np.percentile(df[column].values, 5)
    quantile_2 = np.percentile(df[column].values, 95)
  
    IQR = quantile_2 - quantile_1
    IQR_weight = IQR * weight
  
    lowest = quantile_1 - IQR_weight
  
    outlier_idx = df[column][(df[column] < lowest)].index
    return outlier_idx 
 
# 이상치 제거 (2) : Z-score : 3-sigma 밖의 값 제거
def outliers_z_score(df=None, column=None, threshold=3):
    mean = np.mean(df[column].values)
    std = np.std(df[column].values)
    
    z_scores = np.array([(y-mean)/std for y in df[column].values])
    
    outlier_idx = df[column][(np.abs(z_scores)>threshold)].index
    return outlier_idx

In [11]:
# (1) IQR 활용 Version
right_skewed_list = list(x_drop_all.columns[x_drop_all.skew()>10])
no_skewed_list = list(x_drop_all.columns[np.abs(x_drop_all.skew())<10])
left_skewed_list = list(x_drop_all.columns[x_drop_all.skew()<-10])

outlier_idxes = []

# right_skewed
outlier_idxes_right=[]
for i, col_name in enumerate(right_skewed_list):
    outlier_idx = get_outlier_right_skewed(df=x_drop_all, column=col_name, weight=25)
    print(col_name, '--> 제거된 Outlier 수:',len(outlier_idx))
    outlier_idxes_right.extend(list(outlier_idx))
    #print('\n')
outlier_idxes_right = list(set(outlier_idxes_right))
print('right_skewed Outlier 수 -->', len(outlier_idxes_right))
print('\n')


#no skewed
outlier_idxes_no_skew=[]
for i, col_name in enumerate(no_skewed_list):
    outlier_idx = get_outlier_no_skewed(df=x_drop_all, column=col_name, weight=2)
    print(col_name, '--> 제거된 Outlier 수',len(outlier_idx))
    outlier_idxes_no_skew.extend(list(outlier_idx))
    #print('\n')
outlier_idxes_no_skew = list(set(outlier_idxes_no_skew))
print('no_skewed Outlier 수 -->', len(outlier_idxes_no_skew))
print('\n')

#left skewed
outlier_idxes_left=[]
for i, col_name in enumerate(left_skewed_list):
    outlier_idx = get_outlier_left_skewed(df=x_drop_all, column=col_name, weight=5)
    print(col_name, '--> 제거된 Outlier 수:',len(outlier_idx))
    outlier_idxes_left.extend(list(outlier_idx))
    #print('\n')
outlier_idxes_left = list(set(outlier_idxes_left))
print('left_skewed Outlier 수 -->', len(outlier_idxes_left))

outlier_idxes.extend(outlier_idxes_right)
outlier_idxes.extend(outlier_idxes_no_skew)
outlier_idxes.extend(outlier_idxes_left)

outlier_idxes = list(set(outlier_idxes))
outlier_idxes.sort()

x_iqr = x_drop_all.drop(outlier_idxes, axis=0, inplace=False).reset_index(drop=True)
y_iqr = y_drop_all.drop(outlier_idxes, axis=0, inplace=False).reset_index(drop=True)

print('Outlier 제거 전 : ', len(x_drop_all))
print('Outlier 제거 후 :', len(x_iqr))

x_3 --> 제거된 Outlier 수: 501
x_6 --> 제거된 Outlier 수: 419
x_9 --> 제거된 Outlier 수: 390
x_38 --> 제거된 Outlier 수: 1
x_129 --> 제거된 Outlier 수: 107
x_130 --> 제거된 Outlier 수: 46
x_139 --> 제거된 Outlier 수: 0
x_155 --> 제거된 Outlier 수: 0
x_163 --> 제거된 Outlier 수: 1
x_271 --> 제거된 Outlier 수: 5
x_380 --> 제거된 Outlier 수: 0
x_381 --> 제거된 Outlier 수: 0
x_382 --> 제거된 Outlier 수: 0
x_383 --> 제거된 Outlier 수: 0
x_384 --> 제거된 Outlier 수: 0
x_385 --> 제거된 Outlier 수: 0
x_386 --> 제거된 Outlier 수: 0
right_skewed Outlier 수 --> 670


x_1 --> 제거된 Outlier 수 3
x_2 --> 제거된 Outlier 수 391
x_4 --> 제거된 Outlier 수 0
x_5 --> 제거된 Outlier 수 157
x_7 --> 제거된 Outlier 수 771
x_8 --> 제거된 Outlier 수 1047
x_10 --> 제거된 Outlier 수 0
x_11 --> 제거된 Outlier 수 0
x_12 --> 제거된 Outlier 수 0
x_13 --> 제거된 Outlier 수 0
x_14 --> 제거된 Outlier 수 0
x_15 --> 제거된 Outlier 수 0
x_16 --> 제거된 Outlier 수 0
x_17 --> 제거된 Outlier 수 0
x_18 --> 제거된 Outlier 수 0
x_19 --> 제거된 Outlier 수 0
x_20 --> 제거된 Outlier 수 0
x_21 --> 제거된 Outlier 수 0
x_22 --> 제거된 Outlier 수 0
x_23 --> 제거된 Outlier 수 0
x_2

In [12]:
# # (2) Z-score 활용 Version
# outlier_idxes_z=[]
# for i, col_name in enumerate(x_drop_all.columns):
#     outlier_idx = outliers_z_score(df=x_drop_all, column=col_name, threshold=4)
#     print(col_name, '--> 제거된 Outlier 수',len(outlier_idx))
#     outlier_idxes_z.extend(list(outlier_idx))
#     #print('\n')
    
# outlier_idxes_z = list(set(outlier_idxes_z))
# print('Z-score Outlier 수 -->', len(outlier_idxes_z))
# outlier_idxes_z.sort()

# x_z = x_drop_all.drop(outlier_idxes_z, axis=0, inplace=False).reset_index(drop=True)
# y_z = y_drop_all.drop(outlier_idxes_z, axis=0, inplace=False).reset_index(drop=True)

# print('Outlier 제거 전 : ', len(x_drop_all))
# print('Outlier 제거 후 :', len(x_z))

In [13]:
# 학습 : 검증 : 테스트 = 6 : 3 : 1 데이터 분리
X_train, X_valid, Y_train, Y_valid = train_test_split(x_iqr, y_iqr, test_size=0.4, random_state=2021)
X_vaild, X_test, Y_valid, Y_test = train_test_split(X_valid, Y_valid, test_size=0.25, random_state=2021)

# RobustScaler
Robust_scaler = RobustScaler()
Robust_scaler.fit(X_train)
X_train_RB = Robust_scaler.transform(X_train)
X_test_RB = Robust_scaler.transform(X_test)

# MaxAbsScaler
MA_scaler = MaxAbsScaler()
MA_scaler.fit(X_train)
X_train_MA = MA_scaler.transform(X_train)
X_test_MA = MA_scaler.transform(X_test)

In [14]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

## Modeling 

In [15]:
# # 변수 선택 : 1. 다중공선성 제거
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(x_iqr.values, i) for i in range(x_iqr.shape[1])]
# vif["features"] = x_iqr.columns

# new_columns = list(vif.loc[vif["VIF Factor"]<10]['features'])

# # 학습 : 검증 : 테스트 = 6 : 3 : 1 데이터 분리
# new_X_train, new_X_valid, Y_train, Y_valid = train_test_split(x_iqr[new_columns], y_iqr, test_size=0.4, random_state=2021)
# new_X_vaild, new_X_test, Y_valid, Y_test = train_test_split(new_X_valid, Y_valid, test_size=0.25, random_state=2021)

# # RobustScaler
# Robust_scaler = RobustScaler()
# Robust_scaler.fit(new_X_train)
# new_X_train_RB = Robust_scaler.transform(new_X_train)
# new_X_test_RB = Robust_scaler.transform(new_X_test)

# # MaxAbsScaler
# MA_scaler = MaxAbsScaler()
# MA_scaler.fit(new_X_train)
# new_X_train_MA = MA_scaler.transform(new_X_train)
# new_X_test_MA = MA_scaler.transform(new_X_test)

### After Handle Multicollinearity

#### RandomForest Regressor Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from pprint import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 700, num = 3)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(150, 400, num = 3)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 4, 6]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 3, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth
              }

pprint(random_grid)

rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = GridSearchCV(estimator = rf,
                         param_grid = random_grid,
                         cv = 3
                        )
# Fit the random search model
rf_random_st = rf_random.fit(X_train_RB, Y_train)
rf_random_st.best_params_

{'max_depth': [150, 275, 400, None], 'n_estimators': [200, 450, 700]}


In [ ]:
rf_random_RB = rf_random.fit(X_train_MA, Y_train)
rf_random_RB.best_params_

In [ ]:
# class MLP_RRegressor(nn.Module):
#     def __init__(self, length):
#         super().__init__()
#         self.layer_1 = nn.Linear(in_features,50)
#         self.layer_2 = nn.Linear(50,10)
#         self.layer_out = nn.Linear(10, 1)
        
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.05)
        
#         torch.nn.init.xavier_uniform_(self.layer_1.weight)
#         torch.nn.init.xavier_uniform_(self.layer_2.weight)
        
#     def forward(self, inputs):
#         x = self.relu(self.layer_1(inputs))
#         x = self.relu(self.layer_2(x))
#         x = self.dropout(x)
#         return x

In [ ]:
# train_features = torch.tensor(X_train_RB.values, dtype=torch.float32)
# test_features = torch.tensor(X_test_RB.values, dtype=torch.float32)

# train_labels = torch.tensor(Y_train.values, dtype=torch.float32)
# test_labels = torch.tensor(Y_test.values, dtype=torch.float32)

# loss = nn.MSELoss()

# in_features = train_features.shape[1]
# out_features = 1

# def get_net():
#     net = nn.Sequential(nn.Linear(in_features,out_features))
#     return net